In [18]:
from dotenv import load_dotenv
import getpass
import os

load_dotenv()
gemini_token = os.getenv("GEMINI_TOKEN")
os.environ["TAVILY_API_KEY"] = getpass.getpass()

## Tool

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1722287876, 'localtime': '2024-07-29 14:17'}, 'current': {'last_updated_epoch': 1722287700, 'last_updated': '2024-07-29 14:15', 'temp_c': 17.8, 'temp_f': 64.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 12.1, 'wind_kph': 19.4, 'wind_degree': 260, 'wind_dir': 'W', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 70, 'cloud': 0, 'feelslike_c': 17.8, 'feelslike_f': 64.0, 'windchill_c': 17.8, 'windchill_f': 64.0, 'heatindex_c': 17.8, 'heatindex_f': 64.0, 'dewpoint_c': 12.5, 'dewpoint_f': 54.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 5.0, 'gust_mph': 15.9, 'gust_kph': 25.6}}"}, {'url': 'https://forecast.weather.gov/MapClick.php?x=165

## Model

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=gemini_token)

In [21]:
from langchain_core.messages import HumanMessage

response = chat_model.invoke([HumanMessage(content="HI!")])
response.content

'Hello there! How can I assist you today?'

## Bind Tool

In [22]:
model_with_tools = chat_model.bind_tools(tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [23]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello, how can I help you today?
ToolCalls: []


In [30]:
response = model_with_tools.invoke([HumanMessage(content="What's Messi's age?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavilly_search_results_json', 'args': {'query': "What's Messi's age?"}, 'id': 'd0744a24-2ccf-4c12-ae47-087a8a89b837', 'type': 'tool_call'}]


## Create Agent

In [32]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(chat_model, tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [33]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='20ef24da-f387-497e-afcc-dd1d55612046'),
 AIMessage(content='Hello there! How can I assist you today?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-3d0e5e80-aeca-4a8b-a746-2fe3f41b0043-0', usage_metadata={'input_tokens': 76, 'output_tokens': 10, 'total_tokens': 86})]

In [38]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Weather in SF?")]}
)
response["messages"]

[HumanMessage(content='Weather in SF?', id='78ce1916-3e58-4f5b-a4fc-870b70fc815c'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is the weather in San Francisco today?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-b032e00b-ce78-42d9-97d8-fd6fd4714060-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is the weather in San Francisco today?'}, 'id': '4b5a5a82-8574-40ff-85e0-db02f9f004fc', 'type': 'tool_call'}], usage_meta

## Streaming messages

In [43]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Weather in SF?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "Weather in San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-cb1f4141-b344-4a80-a98c-f820ba133b51-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Weather in San Francisco'}, 'id': '70cee270-b7ac-4fb8-bb2f-c5a25b3f7cd0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 78, 'output_tokens': 23, 'total_tokens': 101})]}}
----
{'tools': {'messages':